In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from itertools import combinations
from random import sample
import seaborn as sns
import networkx as nx 
sns.set_style("darkgrid")
import time


# Load Dataset & Make realizations of graph 

In [ ]:
def get_realizations(real_n=20,real_p=0.005):
    Graphs = []
    Graphs_inv = []
    for _ in range(real_n):
        graph = nx.DiGraph()
        graph_inv = nx.DiGraph()
        dataset = open("./CA-AstroPh.txt")
        for line in dataset :
            if line is None :
                break
            line = str(line)
            if  line.startswith("#"):
                continue
            s,t = map(int,line.split())
            graph.add_node(s)
            graph.add_node(t)
            graph_inv.add_node(s)
            graph_inv.add_node(t)
            if np.random.rand() <= real_p :
                graph.add_edge(s,t)
                graph_inv.add_edge(t,s)
                
        if (_+1)%10 ==0:
            print("Generated Graphs:",_+1)
                
        Graphs.append(graph)
        Graphs_inv.append(graph_inv)
        
    return Graphs,Graphs_inv


# Custom Class

In [ ]:
class GRAPH: 
    def __init__(self,g:nx.DiGraph,is_OutBreak=False) -> None:
        self.graph = g
        self.covers = set()
        self.influence = []
        self.is_OutBreak = is_OutBreak
        
    def get_cost(self,node):
        if not self.is_OutBreak :
            return 1
        
        degree = self.graph.out_degree(node) + self.graph.in_degree(node) 
        return degree
        
        
        
    def get_gain(self,node):

        outs = self.get_outs(node)
        new_outs = [ new_out for new_out in outs if not(new_out in self.covers)]
        if len(new_outs) == 1:
            return 0
        return len(new_outs)
    
    def add_to_set(self,node):
        self.influence.append(node)
        self.covers.add(node)
        outs = self.get_outs(node)
        for out in outs :
            self.covers.add(out)
            
            
    def get_outs(self,node):
        visited = set([node])
        queue = [neighbor for neighbor in self.graph.neighbors(node)]
        while len(queue) != 0 :
            current_node = queue[0]
            visited.add(current_node)
            del queue[0]
            for neighbor in self.graph.neighbors(current_node):
                if not neighbor in visited:
                    queue.append(neighbor)
    
        return visited
        




# Greedy

In [ ]:
def max_influnce_greedy(Graphs,budget=10,is_OutBreak=False,cost=1,gain=1):
    greedy_graphs = []
    greedy_times = []
    costs = []
    for g in Graphs:
        greedy_graphs.append(GRAPH(g.copy(),is_OutBreak))

    Nodes = list(Graphs[0].nodes)

    ref_sec = time.time()
    used_budget = 0
    i = 0
    while used_budget < budget:
        nodes_si = []
        for node in Nodes:
            S_i = 0
            for graph in greedy_graphs:
                reach_nodes = graph.get_gain(node)
                reach_nodes *= gain
                S_i += reach_nodes
                
            nodes_si.append(S_i)
        
        nodes_si = np.array(nodes_si)
        new_node_inf = list(Nodes)[np.argmax(nodes_si)]
        
        sum_cost_on_real = 0
        for graph in greedy_graphs:
            sum_cost_on_real += cost*graph.get_cost(new_node_inf)
        used_budget += sum_cost_on_real/len(greedy_graphs)
        
        if used_budget > budget :
            break
        
        costs.append(sum_cost_on_real/len(greedy_graphs))
        
        for graph in greedy_graphs:
            graph.add_to_set(new_node_inf)
            
        greedy_times.append(time.time()-ref_sec)
        #print("Greedy(Hill Climbing) #{0} Passed".format(i+1))
        i+= 1
    return greedy_graphs,greedy_times,costs
        

# CELF

In [ ]:
def max_influnce_lazy(Graphs,budget=10,is_OutBreak=False,cost=1,gain=1,unit=False):
    costs = []
    celf_graphs = []
    Nodes = list(Graphs[0].nodes)
    Fs = {}
    used_budget = 0
    celf_times = []
    for g in Graphs:
        celf_graphs.append(GRAPH(g.copy(),is_OutBreak))
        
    ref_sec = time.time()
    for node in Nodes:
        S_i = 0
        for graph in celf_graphs:
            reach_nodes = graph.get_gain(node)
            reach_nodes *= gain
            if not unit and reach_nodes != 0:
                reach_nodes /= (cost*graph.get_cost(node))
            S_i += reach_nodes
        Fs[node] = S_i
        
    Fs = {k: v for k, v in sorted(Fs.items(), key=lambda item: -item[1])}
    index = 1
    _k = list(Fs.keys())[0]

    sum_cost_on_real = 0
    for graph in celf_graphs:
        sum_cost_on_real += cost*graph.get_cost(_k)
    used_budget += sum_cost_on_real/len(celf_graphs)
    
    if used_budget > budget :
        print("Error: Can select even one node")
        return celf_graphs,celf_times,costs
    
    for graph in celf_graphs:
            graph.add_to_set(_k)
    del Fs[_k]
    costs.append(sum_cost_on_real/len(celf_graphs))
    
    celf_times.append(time.time()-ref_sec)
    #print("Lazy Hill Climbing #{0} Passed".format(index))
    while used_budget < budget:
        keys = list(Fs.keys())
        _k0,_k1 = keys[0],keys[1] 
        _inf0,_inf1 = Fs[_k0],Fs[_k1]
        
        S_i = 0
        for graph in celf_graphs:
            reach_nodes = graph.get_gain(_k0)
            reach_nodes *= gain
            if not unit  and reach_nodes != 0:
                try: 
                    reach_nodes /= (cost*graph.get_cost(_k0))
                except:
                    print("HERE <*****************************")
                    pass
                    
            S_i += reach_nodes
                        
        if S_i >= _inf1 :
            sum_cost_on_real = 0
            for graph in celf_graphs:
                sum_cost_on_real += cost*graph.get_cost(_k0)
            used_budget += sum_cost_on_real/len(celf_graphs)
            
            if used_budget > budget:
                return celf_graphs,celf_times,costs
            
            for graph in celf_graphs:
                graph.add_to_set(_k0)
            del Fs[_k0]
            costs.append(sum_cost_on_real/len(celf_graphs))
            
            celf_times.append(time.time()-ref_sec)
            index += 1
            #print("Lazy Hill Climbing #{0} Passed".format(index))
            continue
        else:
            Fs[_k0] = S_i
            
        Fs = {k: v for k, v in sorted(Fs.items(), key=lambda item: -item[1])}
    return celf_graphs,celf_times,costs
        


# Main

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)
def report(graphs:list,times:list,costs:list,is_im=False,Name="",p=0):
    assert len(graphs) == len(times) or len(costs) == len(times)
    spread = 0
    for graph in graphs:
        spread += len(graph.covers)
    spread /= len(graphs)
    S = round(sum(costs),2)
    if is_im :
        Name = Name + " \nMean Spread in realizations({0}):{1} \nProb. of active edge:{2}\nSum Cost:{3}".format(len(graphs),spread,p,S)
    else:
        Name = Name + " \nMinimized number of infected people in realizations({0}):{1} \nProb. of active edge:{2}\nSum Cost:{3}".format(len(graphs),spread,p,S)
        
    print("**************************************************************************************************************\nReporting for",Name)
    for index in range(len(costs)):
        print("Time to select node #{0}(ID:{3}) is {1} second and cost of it is equal to {2}".format(index+1,round(times[index],2),round(costs[index],2),graphs[0].influence[index]))
        
        
        

real_n = 10
real_p = 0.01

Graphs,Graphs_inv = get_realizations(real_n,real_p)
Graphs,Graphs_inv = Graphs_inv,Graphs
_greedy_graphs,_greedy_times,_greedy_costs = max_influnce_greedy(Graphs,10,True,0.8,0.25)

_celf_graphs1,_celf_times1,_celf_costs1 = max_influnce_lazy(Graphs,10,True,0.8,0.25,True)
_celf_graphs2,_celf_times2,_celf_costs2 = max_influnce_lazy(Graphs,10,True,0.8,0.25,False)

report(_greedy_graphs,_greedy_times,_greedy_costs,True,"Greedy",real_p)
report(_celf_graphs1,_celf_times1,_celf_costs1,False,"CELF: unit-cost greedy",real_p)
report(_celf_graphs2,_celf_times2,_celf_costs2,False,"CELF: benefit-cost greedy",real_p)

